In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D, Parsers as P, Utilities as U
from mppi.Calculators import Tools
from mppi.Datasets import PostProcessing as PP
from mppi.Utilities import Constants as Const
from mppi.Utilities import LatticeUtils as LL
import matplotlib.pyplot as plt
import numpy as np
import os, yaml

In [3]:
# RunRules for local computations
mpi = 4
omp = 2

rr = C.RunRules(omp_num_threads=omp,mpi=mpi)
code = C.QeCalculator(rr)
#code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler direct


In [3]:
# RunRules for ismhpc
# The product of ntasks_per_node*cpus_per_task is equal to 32. 
# Many mpi are needed for better performances
nodes = 1
ntasks_per_node = 16
cpus_per_task=2
omp_num_threads=2

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='all12h',
                memory='125000',time='11:59:00',
                nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads)
code = C.QeCalculator(rr,activate_BeeOND=True) #,skip=False,clean_restart=False
code.global_options()

Initialize a QuantumESPRESSO calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 16,
 'cpus_per_task': 2,
 'omp_num_threads': 2,
 'gpus_per_node': None,
 'memory': '125000',
 'time': '11:59:00',
 'partition': 'all12h',
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'executable': 'pw.x',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': True,
 'verbose': True}

# PHONON analysis of the bulk WSe$_2$

We compute the electron-phonon matrix elements using the QuantumESPRESSO package.

In [4]:
run_dir = 'PHONON'

## SCF and NSCF calculations

In [5]:
prefix = 'WSe2'
inp = I.PwInput(file='Pw_input_Marini/scf.in')
inp.set_scf(conv_thr=1e-15,force_symmorphic=True)
inp.set_pseudo_dir('../../pseudos')
#print(inp.convert_string())

In [6]:
name = 'WSe2.scf'
scf_run_dir = os.path.join(run_dir,'scf')
results = code.run(input=inp,run_dir=scf_run_dir,name=name)
results

Skip the run of WSe2.scf


'/work/dalessandro/NEQ_EXCITONIC_TRANSITIONS/WSe2/PHONON/scf/WSe2.save/data-file-schema.xml'

In [7]:
nbnds = 70
inp.set_nscf(nbnds,force_symmorphic=True,conv_thr=1e-15)
inp.set_kpoints(points=[9,9,1])
print(inp.convert_string())

&control
         calculation = 'nscf'
              outdir = './'
              prefix = 'WSe2'
          pseudo_dir = '../../pseudos'
           verbosity = 'high'
/&end
&system
           celldm(1) = 6.1983017192617
           celldm(3) = 3.95
             ecutwfc = 50
    force_symmorphic = .true.
               ibrav = 4
            lspinorb = .true.
                 nat = 6
                nbnd = 70
            noncolin = .true.
                ntyp = 2
/&end
&electrons
            conv_thr = 1e-15
      diago_full_acc = .false.
/&end
ATOMIC_SPECIES
  Se   32.065            Se_FR.upf
   W   183.84             W_FR.upf
ATOMIC_POSITIONS { crystal }
  W   0.3333330000   0.6666670000   0.2500000000
  W   0.6666670000   0.3333330000   0.7500000000
 Se   0.3333330000   0.6666670000   0.6211484930
 Se   0.6666670000   0.3333330000   0.1211484930
 Se   0.3333330000   0.6666670000   0.8788515070
 Se   0.6666670000   0.3333330000   0.3788515070
K_POINTS { automatic }
  9  9  1  0  0  0


In [8]:
name = 'WSe2.nscf'
nscf_run_dir = os.path.join(run_dir,'nscf')
results = code.run(input=inp,run_dir=nscf_run_dir,name=name,source_dir=os.path.join(scf_run_dir,prefix+'.save'))
results

Skip the run of WSe2.nscf
The folder /work/dalessandro/NEQ_EXCITONIC_TRANSITIONS/WSe2/PHONON/nscf/WSe2.save already exists. Source_dir PHONON/scf/WSe2.save not copied


'/work/dalessandro/NEQ_EXCITONIC_TRANSITIONS/WSe2/PHONON/nscf/WSe2.save/data-file-schema.xml'

In [9]:
data = P.PwParser(results)

Parse file : /work/dalessandro/NEQ_EXCITONIC_TRANSITIONS/WSe2/PHONON/nscf/WSe2.save/data-file-schema.xml


In [10]:
kpoints=data.kpoints
kpoints

array([[0.        , 0.        , 0.        ],
       [0.        , 0.12830006, 0.        ],
       [0.        , 0.25660012, 0.        ],
       [0.        , 0.38490018, 0.        ],
       [0.        , 0.51320024, 0.        ],
       [0.11111111, 0.19245009, 0.        ],
       [0.11111111, 0.32075015, 0.        ],
       [0.11111111, 0.44905021, 0.        ],
       [0.11111111, 0.57735027, 0.        ],
       [0.22222222, 0.38490018, 0.        ],
       [0.22222222, 0.51320024, 0.        ],
       [0.33333333, 0.57735027, 0.        ]])

## Phonon calculations

We run the computation to compute the dynamical matrix and the dvscf files. Computations are very lengthly and have
been split into three groups, following the approach of Andrea.

In [20]:
nodes = 2
ntasks_per_node = 32
cpus_per_task=1
omp_num_threads=1

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='slownodes',
                memory='125000',nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads)
code_ph = C.QeCalculator(rr,executable='ph.x',activate_BeeOND=True) #,skip=False,clean_restart=False
code_ph.global_options()

Initialize a QuantumESPRESSO calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 2,
 'ntasks_per_node': 32,
 'cpus_per_task': 1,
 'omp_num_threads': 1,
 'gpus_per_node': None,
 'memory': '125000',
 'time': None,
 'partition': 'slownodes',
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'executable': 'ph.x',
 'skip': True,
 'clean_restart': False,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': True,
 'verbose': True}

In [21]:
# tr_ph=1e-12 has run without reaching convergence
prefix = 'WSe2'
start_q,last_q=1,4
inp = I.PhInput(tr2_ph=5e-9,prefix=prefix)
inp.set_inputph_variables(search_sym='.false.',epsil='.true.',
        fildvscf="'WSe2-dvscf'",fildyn="'WSe2.dyn'",electron_phonon="'dvscf'")
inp.set_inputph_variables(START_Q=start_q,LAST_Q=last_q)
inp.set_kpoints(Tools.build_pw_klist(kpoints))
print(inp.convert_string())

&inputph
              LAST_Q = 4
             START_Q = 1
     electron_phonon = 'dvscf'
               epsil = .true.
            fildvscf = 'WSe2-dvscf'
              fildyn = 'WSe2.dyn'
               ldisp = .false.
              outdir = './'
              prefix = 'WSe2'
               qplot = .true.
          search_sym = .false.
              tr2_ph = 1e-12
               trans = .true.
/&end
12
  0.00000000   0.00000000   0.00000000 1 
  0.00000000   0.12830006   0.00000000 1 
  0.00000000   0.25660012   0.00000000 1 
  0.00000000   0.38490018   0.00000000 1 
  0.00000000   0.51320024   0.00000000 1 
  0.11111111   0.19245009   0.00000000 1 
  0.11111111   0.32075015   0.00000000 1 
  0.11111111   0.44905021   0.00000000 1 
  0.11111111   0.57735027   0.00000000 1 
  0.22222222   0.38490018   0.00000000 1 
  0.22222222   0.51320024   0.00000000 1 
  0.33333333   0.57735027   0.00000000 1 


In [ ]:
name = 'WSe2-startq_%s-lastq_%s.phonon'%(start_q,last_q)
ph_run_dir = os.path.join(run_dir,'phonon-startq_%s-lastq_%s'%(start_q,last_q))
results = code_ph.run(input=inp,run_dir=ph_run_dir,name=name,source_dir=os.path.join(scf_run_dir,prefix+'.save'))

create the run_dir folder : 'PHONON/phonon-startq_1-lastq_4'
run performed starting from existing results
copy source_dir PHONON/scf/WSe2.save in the /work/dalessandro/NEQ_EXCITONIC_TRANSITIONS/WSe2/PHONON/phonon-startq_1-lastq_4/WSe2.save
run command: mpirun -np 64 ph.x -inp WSe2-startq_1-lastq_4.phonon.in > WSe2-startq_1-lastq_4.phonon.log
slurm submit:  cd PHONON/phonon-startq_1-lastq_4 ; sbatch job_WSe2-startq_1-lastq_4.phonon.sh
computation WSe2-startq_1-lastq_4.phonon is running...


In [15]:
prefix = 'WSe2'
start_q,last_q=5,8
inp = I.PhInput(tr2_ph=1e-12,prefix=prefix)
inp.set_inputph_variables(search_sym='.false.',epsil='.false.',
        fildvscf="'WSe2-dvscf'",fildyn="'WSe2.dyn'",electron_phonon="'dvscf'")
inp.set_inputph_variables(START_Q=start_q,LAST_Q=last_q)
inp.set_kpoints(Tools.build_pw_klist(kpoints))
print(inp.convert_string())

&inputph
              LAST_Q = 8
             START_Q = 5
     electron_phonon = 'dvscf'
               epsil = .false.
            fildvscf = 'WSe2-dvscf'
              fildyn = 'WSe2.dyn'
               ldisp = .false.
              outdir = './'
              prefix = 'WSe2'
               qplot = .true.
          search_sym = .false.
              tr2_ph = 1e-12
               trans = .true.
/&end
12
  0.00000000   0.00000000   0.00000000 1 
  0.00000000   0.12830006   0.00000000 1 
  0.00000000   0.25660012   0.00000000 1 
  0.00000000   0.38490018   0.00000000 1 
  0.00000000   0.51320024   0.00000000 1 
  0.11111111   0.19245009   0.00000000 1 
  0.11111111   0.32075015   0.00000000 1 
  0.11111111   0.44905021   0.00000000 1 
  0.11111111   0.57735027   0.00000000 1 
  0.22222222   0.38490018   0.00000000 1 
  0.22222222   0.51320024   0.00000000 1 
  0.33333333   0.57735027   0.00000000 1 


In [ ]:
name = 'WSe2-startq_%s-lastq_%s.phonon'%(start_q,last_q)
ph_run_dir = os.path.join(run_dir,'phonon-startq_%s-lastq_%s'%(start_q,last_q))
results = code_ph.run(input=inp,run_dir=ph_run_dir,name=name,source_dir=os.path.join(scf_run_dir,prefix+'.save'))

In [17]:
prefix = 'WSe2'
start_q,last_q=9,12
inp = I.PhInput(tr2_ph=1e-12,prefix=prefix)
inp.set_inputph_variables(search_sym='.false.',epsil='.false.',
        fildvscf="'WSe2-dvscf'",fildyn="'WSe2.dyn'",electron_phonon="'dvscf'")
inp.set_inputph_variables(START_Q=start_q,LAST_Q=last_q)
inp.set_kpoints(Tools.build_pw_klist(kpoints))
print(inp.convert_string())

&inputph
              LAST_Q = 12
             START_Q = 9
     electron_phonon = 'dvscf'
               epsil = .false.
            fildvscf = 'WSe2-dvscf'
              fildyn = 'WSe2.dyn'
               ldisp = .false.
              outdir = './'
              prefix = 'WSe2'
               qplot = .true.
          search_sym = .false.
              tr2_ph = 1e-12
               trans = .true.
/&end
12
  0.00000000   0.00000000   0.00000000 1 
  0.00000000   0.12830006   0.00000000 1 
  0.00000000   0.25660012   0.00000000 1 
  0.00000000   0.38490018   0.00000000 1 
  0.00000000   0.51320024   0.00000000 1 
  0.11111111   0.19245009   0.00000000 1 
  0.11111111   0.32075015   0.00000000 1 
  0.11111111   0.44905021   0.00000000 1 
  0.11111111   0.57735027   0.00000000 1 
  0.22222222   0.38490018   0.00000000 1 
  0.22222222   0.51320024   0.00000000 1 
  0.33333333   0.57735027   0.00000000 1 


In [ ]:
name = 'WSe2-startq_%s-lastq_%s.phonon'%(start_q,last_q)
ph_run_dir = os.path.join(run_dir,'phonon-startq_%s-lastq_%s'%(start_q,last_q))
results = code_ph.run(input=inp,run_dir=ph_run_dir,name=name,source_dir=os.path.join(scf_run_dir,prefix+'.save'))

delete job_out script: PHONON/phonon-startq_9-lastq_12/job_WSe2-startq_9-lastq_12.phonon.out
run performed starting from existing results
The folder /work/dalessandro/NEQ_EXCITONIC_TRANSITIONS/WSe2/PHONON/phonon-startq_9-lastq_12/WSe2.save already exists. Source_dir PHONON/scf/WSe2.save not copied
run command: mpirun -np 32 ph.x -inp WSe2-startq_9-lastq_12.phonon.in > WSe2-startq_9-lastq_12.phonon.log
slurm submit:  cd PHONON/phonon-startq_9-lastq_12 ; sbatch job_WSe2-startq_9-lastq_12.phonon.sh
computation WSe2-startq_9-lastq_12.phonon is running...


## Electron-phonon calculations

We compute the electron-phonon matrix elements.

In [11]:
elph_run_dir = 'PHONON/elph'
elph_path = 'PHONON/elph/_ph0/WSe2.phsave'
if not os.path.isdir(elph_path) : os.makedirs(elph_path)

In [13]:
ph_dirs = ['PHONON/phonon-startq_1-lastq_4','PHONON/phonon-startq_5-lastq_8','PHONON/phonon-startq_9-lastq_12']

In [14]:
# copy the dyn files
for d in ph_dirs: 
    os.system('cp %s/WSe2.dyn* PHONON/elph/'%d)

In [15]:
# copy the dvscf files
for d in ph_dirs: 
    os.system('cp %s/_ph0/WSe2.WSe2-dvscf* PHONON/elph/_ph0/'%d)

In [16]:
# copy the content of the WSe2.phsave
for d in ph_dirs: 
    os.system('cp %s/_ph0/WSe2.phsave/* PHONON/elph/_ph0/WSe2.phsave/'%d)

In [17]:
# copy the nscf in the elph
os.system('cp -r PHONON/nscf/WSe2.save/ PHONON/elph/')

0

In [18]:
# copy the nscf in the elph/_ph0
os.system('cp -r PHONON/nscf/WSe2.save/ PHONON/elph/_ph0/')

0

In [19]:
nodes = 1
ntasks_per_node = 32
cpus_per_task=1
omp_num_threads=1

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='slownodes',
                memory='125000',nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads)
code_ph = C.QeCalculator(rr,executable='ph.x',activate_BeeOND=False,clean_restart=False) #,skip=False,clean_restart=False
code_ph.global_options()

Initialize a QuantumESPRESSO calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 32,
 'cpus_per_task': 1,
 'omp_num_threads': 1,
 'gpus_per_node': None,
 'memory': '125000',
 'time': None,
 'partition': 'slownodes',
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'executable': 'ph.x',
 'skip': True,
 'clean_restart': False,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True}

In [20]:
prefix = 'WSe2'
#start_q,last_q=1,4
inp = I.PhInput(tr2_ph=1e-12,prefix=prefix)
inp.set_inputph_variables(epsil='.false.',trans='.false.',
        fildvscf="'WSe2-dvscf'",fildyn="'WSe2.dyn'",electron_phonon="'yambo'")
#inp.set_inputph_variables(START_Q=start_q,LAST_Q=last_q)
inp.set_kpoints(Tools.build_pw_klist(kpoints))
print(inp.convert_string())

&inputph
     electron_phonon = 'yambo'
               epsil = .false.
            fildvscf = 'WSe2-dvscf'
              fildyn = 'WSe2.dyn'
               ldisp = .false.
              outdir = './'
              prefix = 'WSe2'
               qplot = .true.
              tr2_ph = 1e-12
               trans = .false.
/&end
12
  0.00000000   0.00000000   0.00000000 1 
  0.00000000   0.12830006   0.00000000 1 
  0.00000000   0.25660012   0.00000000 1 
  0.00000000   0.38490018   0.00000000 1 
  0.00000000   0.51320024   0.00000000 1 
  0.11111111   0.19245009   0.00000000 1 
  0.11111111   0.32075015   0.00000000 1 
  0.11111111   0.44905021   0.00000000 1 
  0.11111111   0.57735027   0.00000000 1 
  0.22222222   0.38490018   0.00000000 1 
  0.22222222   0.51320024   0.00000000 1 
  0.33333333   0.57735027   0.00000000 1 


In [ ]:
name = 'WSe2'
results = code_ph.run(input=inp,run_dir=elph_run_dir,name=name)

run performed starting from existing results
run command: mpirun -np 32 ph.x -inp WSe2.in > WSe2.log
slurm submit:  cd PHONON/elph ; sbatch job_WSe2.sh
computation WSe2 is running...


Ph.x cannot compute the elph with th non collinear spin-orbit. 

I have removed this check in the
line 820 of the file phq_reading.f90 (see https://gitlab.com/QEF/q-e/-/blob/fb444989adf93e51b983e53a44d5e510e7d7c668/PHonon/PH/phq_readin.f90).

## Import in Yambo

We use ypp_ph to generate the electron-phonon database that can be used by yambo.

This procedure convert the file written in the elph/elph_dir

In [14]:
source_dir = 'PHONON/nscf/WSe2.save'
run_dir = 'PHONON/yambo'
Tools.init_yambo_run_dir(Tools.make_p2y(source_dir),run_dir=run_dir)

Executing command: cd PHONON/nscf/WSe2.save; p2y
Create folder path PHONON/yambo
Create a symlink of PHONON/nscf/WSe2.save/SAVE in PHONON/yambo
Build the r_setup in the run_dir path PHONON/yambo


In [16]:
elph_dir = 'PHONON/elph/elph_dir'
if not os.path.isdir('PHONON/yambo/elph_dir'):
    src = os.path.abspath(elph_dir)
    dest = os.path.abspath('PHONON/yambo/elph_dir')
    os.symlink(src,dest,target_is_directory=True)

In [17]:
nodes = 1
ntasks_per_node = 32
cpus_per_task=1
omp_num_threads=1

ntasks = nodes*ntasks_per_node

rr = C.RunRules(scheduler='slurm',partition='all12h',
                memory='125000',nodes=nodes,ntasks_per_node=ntasks_per_node,
                cpus_per_task=cpus_per_task,
                omp_num_threads=omp_num_threads)
code = C.YamboCalculator(rr,executable='ypp_ph')
code.global_options()

Initialize a Yambo calculator with scheduler slurm


{'scheduler': 'slurm',
 'nodes': 1,
 'ntasks_per_node': 32,
 'cpus_per_task': 1,
 'omp_num_threads': 1,
 'gpus_per_node': None,
 'memory': '125000',
 'time': None,
 'partition': 'all12h',
 'account': None,
 'qos': None,
 'map_by': None,
 'pe': 1,
 'rank_by': None,
 'executable': 'ypp_ph',
 'skip': True,
 'clean_restart': True,
 'dry_run': False,
 'wait_end_run': True,
 'activate_BeeOND': False,
 'verbose': True,
 'fatlog': False}

In [21]:
inp = I.YamboInput(args='ypp_ph -g g',folder=run_dir)
inp.set_scalar_variables(DBsPATH='elph_dir/')
inp['arguments'].append('GkkpExpand')
inp

{'args': 'ypp_ph -g g',
 'folder': 'PHONON/yambo',
 'filename': 'yambo.in',
 'arguments': ['gkkp', 'GkkpExpand'],
 'variables': {'PHfreqF': 'none', 'PHmodeF': 'none', 'DBsPATH': 'elph_dir/'}}

In [22]:
inp.write('PHONON/yambo/','GKKP_expanded.in')

In [ ]:
name= 'GKKP_expanded'
code.run(input=inp,name=name,run_dir=run_dir)